# 3.10 時系列データの扱い

時系列データの種類や性質，注意点，データの操作について：3.10.1~3.10.3

時系列データから特徴量を作成する方法について：3.10.4~3.10.5

使えるデータの期間についてさらに注意が必要な点：3.10.6

## 3.10.1 時系列データとは？

時系列データ：時間の推移とともに観測されたデータ

どのような時系列データがあるかによって分析の方針や注意点が変わってくる．以下では3つのケースを考える．

### ケースa: 単に時間情報を持つ変数があるだけの場合

「3.9.2 時間的な統計量をとる」にあるように，時間の情報量をうまく使って特徴量を作ることになる．
分析コンペのタスクとしては，例えば以下のようなもの．

- ユーザの属性や過去の行動ログが与えられる
- 予測対称は1ヶ月以内に解約するかどうか
- **ある時点のユーザを分割して**，学習データ・テストデータが作成されている

**ユーザの行動ログ**

| ユーザID | 日時 | イベント | （その他のイベント情報） |
|:-:|:-:|:-:|:-:|
| 1996 | 2018/1/1 XX:XX:XX | ログイン | ... |
| 1996 | 2018/1/1 XX:XX:XX | サービス利用 | ... |
| 7 | 2018/1/1 XX:XX:XX | ログイン | ... |
| 7 | 2018/1/1 XX:XX:XX | 課金 | ... |
| 7 | 2018/1/2 XX:XX:XX | サービス利用 | ... |
| 7 | 2018/1/2 XX:XX:XX | サービス利用 | ... |
| ... | ... | ... | ... |
| 1 | 2018/12/31 XX:XX:XX | サービス利用 | ... |
| 1 | 2018/12/31 XX:XX:XX | サービス利用 | ... |
| 1 | 2018/12/31 XX:XX:XX | サービス利用 | ... |
| 1 | 2018/12/31 XX:XX:XX | サービス利用 | ... |
| 11 | 2018/12/31 XX:XX:XX | ログイン | ... |
| 11 | 2018/12/31 XX:XX:XX | 課金 | ... |
| 11 | 2018/12/31 XX:XX:XX | サービス利用 | ... |

**学習データ**

| ユーザID | 性別 | 年齢 | （その他のユーザ属性） | 目的変数（1か月以内に解約） |
|:-:|:-:|:-:|:-:|:-:|
| 1 | M | 42 | ... | 0 |
| 2 | F | 34 | ... | 1 |
| 3 | M | 5 | ... | 1 |
| ... | ... | ... | ... | ... |
| 999 | M | 10 | ... | 0 |
| 1000 | F | 54 | ... | 0 |

**テストデータ**

| ユーザID | 性別 | 年齢 | （その他のユーザ属性） |
|:-:|:-:|:-:|:-:|
| 1001 | F | 20 | ... |
| 1002 | F | 25 | ... |
| 1003 | M | 21 | ... |
| ... | ... | ... | ... |
| 1999 | F | 37 | ... |
| 2000 | M | 29 | ... |

### ケースb: ケースaに加えて，学習データ・テストデータが時系列で別れているなど，時間に沿って分割したバリデーションが必要な場合

将来の情報を不適切に使わないように気をつけ，時間に沿って分割したバリデーションを行う（「5.3 時系列データのバリデーション手法」で解説）．
分析コンペのタスクとしては，例えば以下のようなもの

- ユーザの属性や過去の行動ログが与えられる
- 予測対称は1ヶ月以内に解約するかどうか
- 学習データとして各月ごとに月初に存在するユーザとその月内に退会したかどうかが与えられ，テストデータは学習データにない時点のデータが与えられる

**ユーザの行動ログ**

| ユーザID | 日時 | イベント | （その他のイベント情報） |
|:-:|:-:|:-:|:-:|
| 1996 | 2018/1/1 XX:XX:XX | ログイン | ... |
| 1996 | 2018/1/1 XX:XX:XX | サービス利用 | ... |
| 7 | 2018/1/1 XX:XX:XX | ログイン | ... |
| 7 | 2018/1/1 XX:XX:XX | 課金 | ... |
| 7 | 2018/1/2 XX:XX:XX | サービス利用 | ... |
| 7 | 2018/1/2 XX:XX:XX | サービス利用 | ... |
| ... | ... | ... | ... |
| 1 | 2018/12/31 XX:XX:XX | サービス利用 | ... |
| 1 | 2018/12/31 XX:XX:XX | サービス利用 | ... |
| 1 | 2018/12/31 XX:XX:XX | サービス利用 | ... |
| 1 | 2018/12/31 XX:XX:XX | サービス利用 | ... |
| 11 | 2018/12/31 XX:XX:XX | ログイン | ... |
| 11 | 2018/12/31 XX:XX:XX | 課金 | ... |
| 11 | 2018/12/31 XX:XX:XX | サービス利用 | ... |

**ユーザの属性**

| ユーザID | 性別 | 年齢 | （その他のユーザ属性） | 
|:-:|:-:|:-:|:-:|
| 1 | M | 42 | ... |
| 2 | F | 34 | ... |
| 3 | M | 5 | ... |
| 4 | M | 10 | ... |
| 5 | F | 54 | ... |
| ... | ... | ... | ... |
| 1996 | F | 20 | ... |
| 1997 | F | 25 | ... |
| 1998 | M | 21 | ... |
| 1999 | F | 37 | ... |
| 2000 | M | 29 | ... |

**学習データ**

| ユーザID | 対象年月 | 目的変数（1か月以内に解約） |
|:-:|:-:|:-:|
| 1 | 2018/1 | 0 |
| 1 | 2018/2 | 0 |
| 1 | ... | 0 |
| 1 | 2018/12 | 0 |
| 2 | 2018/9 | 0 |
| 2 | 2018/10 | 1 |
| ... | ... | ... |
| 1999 | 2018/1 | 0 |
| 1999 | 2018/2 | 0 |
| 1999 | 2018/3 | 1 |
| 2000 | 2018/11 | 0 |
| 2000 | 2018/12 | 0 |

**テストデータ**

| ユーザID | 対象年月 |
|:-:|:-:|
| 1 | 2019/1 |
| 2 | 2019/1 |
| 3 | 2019/1 |
| ... | ... | ... |
| 1999 | 2019/1 |
| 2000 | 2019/1 |

### ケースc: ケースbに加えて，ユーザや店舗といった系列ごとに時系列の目的変数があり，ラグ特徴量がとれる形式の場合

将来の情報を不適切に使わないように気をつけ，時間に沿って分割したバリデーションを行う（「5.3 時系列データのバリデーション手法」で解説）．
分析コンペのタスクとしては，例えば以下のようなもの

- ユーザの属性や過去の行動ログが与えられる
- 予測対称は1ヶ月以内に解約するかどうか
- 学習データとして各月ごとに月初に存在するユーザとその月内に退会したかどうかが与えられ，テストデータは学習データにない時点のデータが与えられる

**ユーザの行動ログ**

| ユーザID | 日時 | イベント | （その他のイベント情報） |
|:-:|:-:|:-:|:-:|
| 1996 | 2018/1/1 XX:XX:XX | ログイン | ... |
| 1996 | 2018/1/1 XX:XX:XX | サービス利用 | ... |
| 7 | 2018/1/1 XX:XX:XX | ログイン | ... |
| 7 | 2018/1/1 XX:XX:XX | 課金 | ... |
| 7 | 2018/1/2 XX:XX:XX | サービス利用 | ... |
| 7 | 2018/1/2 XX:XX:XX | サービス利用 | ... |
| ... | ... | ... | ... |
| 1 | 2018/12/31 XX:XX:XX | サービス利用 | ... |
| 1 | 2018/12/31 XX:XX:XX | サービス利用 | ... |
| 1 | 2018/12/31 XX:XX:XX | サービス利用 | ... |
| 1 | 2018/12/31 XX:XX:XX | サービス利用 | ... |
| 11 | 2018/12/31 XX:XX:XX | ログイン | ... |
| 11 | 2018/12/31 XX:XX:XX | 課金 | ... |
| 11 | 2018/12/31 XX:XX:XX | サービス利用 | ... |

**ユーザの属性**

| ユーザID | 性別 | 年齢 | （その他のユーザ属性） | 
|:-:|:-:|:-:|:-:|
| 1 | M | 42 | ... |
| 2 | F | 34 | ... |
| 3 | M | 5 | ... |
| 4 | M | 10 | ... |
| 5 | F | 54 | ... |
| ... | ... | ... | ... |
| 1996 | F | 20 | ... |
| 1997 | F | 25 | ... |
| 1998 | M | 21 | ... |
| 1999 | F | 37 | ... |
| 2000 | M | 29 | ... |

**学習データ**

| ユーザID | 対象年月 | 目的変数（1か月以内に解約） |
|:-:|:-:|:-:|
| 1 | 2018/1 | 0 |
| 1 | 2018/2 | 0 |
| 1 | ... | 0 |
| 1 | 2018/12 | 0 |
| 2 | 2018/9 | 0 |
| 2 | 2018/10 | 1 |
| ... | ... | ... |
| 1999 | 2018/1 | 0 |
| 1999 | 2018/2 | 0 |
| 1999 | 2018/3 | 1 |
| 2000 | 2018/11 | 0 |
| 2000 | 2018/12 | 0 |

**テストデータ**

| ユーザID | 対象年月 |
|:-:|:-:|
| 1 | 2019/1 |
| 2 | 2019/1 |
| 3 | 2019/1 |
| ... | ... | ... |
| 1999 | 2019/1 |
| 2000 | 2019/1 |

## 3.10.2 予測するより過去の情報のみを使う

うっかりやってしまわないように注意．
例えば，ケースbの場合でバリデーションより先のデータを学習で使ってしまうと，
バリデーション時点より先の目的変数（解約したかどうか）が大きなヒントになってしまう．
目的変数以外のデータがリークを起こす可能性もあり，
例えばケースbの場合ではユーザの行動ログの中身を見ずに，単にどの時点まで残っているかで解約したかどうかの大きなヒントになる．

以下のような状況では過去の情報も多少使用することが考えられるが，注意は必要である．
バリデーションデータに対して有利な予測をし過ぎている場合はテストデータに対する評価であるPublic Leaderboardのスコアが落ちるのでこれを参考にする．
- 時系列的な性質が弱く，目的変数が過去の目的変数の情報をあまり含まない場合
- 特徴量作成に使うデータがその性質から考えて過去の目的の情報をあまり含まない場合
- データが不足しており，リークの危険性よりも十分なデータで学習することを優先したい場合

